**main**

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image
import numpy as np
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
print("Drive is Ok..........")

Mounted at /content/gdrive
Drive is Ok..........


In [ ]:
# !echo -e 'path = /content/gdrive/MyDrive/YOLOV8\ntrain = /content/images\nval = /content/images\nnames: \n  0: build\n  1: pool' > config.yaml
# print("Ok..........")

In [ ]:
import yaml
config = {'path': '/content/gdrive/MyDrive/YOLOV8',
          'train': '/content/gdrive/MyDrive/YOLOV8/images',
          'val': '/content/gdrive/MyDrive/YOLOV8/images',
          'cn':2,
          'names': ['build', 'Pool']
         }

In [ ]:
with open("config.yaml", "w") as file:
   yaml.dump(config, file, default_flow_style=False)

In [ ]:
!scp -r /content/config.yaml '/content/gdrive/MyDrive/YOLOV8'

In [ ]:
# !rm -rf "/content/images"

In [ ]:
# !mkdir data/obj
!unzip /content/gdrive/MyDrive/YOLOV8/images.zip -d images
print("Ok..........")

In [ ]:
# !scp -r /content/images '/content/gdrive/MyDrive/YOLOV8'
print("Ok..........")

Ok..........


In [ ]:
# !yolo detect train model=yolov8s.pt data=/content/gdrive/MyDrive/YOLOV8/config.yaml epochs=25 imgsz=224 batch=16 plots=True
# !yolo detect train model=yolov8l.pt data=/content/gdrive/MyDrive/YOLOV8/config.yaml epochs=25 imgsz=224 batch=16 plots=True
!yolo detect train model=yolov8x.pt data=/content/gdrive/MyDrive/YOLOV8/config.yaml epochs=100 imgsz=224 batch=16 plots=True

In [ ]:
!scp -r /content/runs/detect/train/weights/best.pt '/content/gdrive/MyDrive/YOLOV8/runs/detect'

In [ ]:
model = YOLO("/content/gdrive/MyDrive/YOLOV8/runs/detect/best.pt")
results = model.predict('/content/gdrive/MyDrive/YOLOV8/images/images/1 (2).JPG')
result = results[0]
output = []
for box in result.boxes:
    x1, y1, x2, y2 = [
       round(x) for x in box.xyxy[0].tolist()
    ]
    class_id = box.cls[0].item()
    prob = round(box.conf[0].item(), 2)
    output.append([
    x1, y1, x2, y2, result.names[class_id], prob
    ])


image 1/1 /content/gdrive/MyDrive/YOLOV8/images/images/1 (2).JPG: 224x224 1 build, 1 Pool, 24.5ms
Speed: 1.2ms preprocess, 24.5ms inference, 2.8ms postprocess per image at shape (1, 3, 224, 224)


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("/content/gdrive/MyDrive/YOLOV8/runs/detect/best.pt")
  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model(['/content/1.jpg'])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs


0: 128x224 1 Pool, 10.1ms
Speed: 0.6ms preprocess, 10.1ms inference, 1.4ms postprocess per image at shape (1, 3, 128, 224)


In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO("/content/gdrive/MyDrive/YOLOV8/runs/detect/best.pt")


# Run inference on an image
results = model('/content/gdrive/MyDrive/YOLOV8/images/images/1 (2).JPG') # results list

# View results
for r in results:
    print(r.boxes)
  # print the Boxes object containing the detection bounding boxes

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

%matplotlib inline

In [ ]:
# Load a pretrained YOLOv8n model
model = YOLO(r"best.pt")

Path = r'2.jpg'
# Run inference on an image
results = model.predict(Path) # results list

# View results
for r in results:
    print(r.boxes)
  # print the Boxes object containing the detection bounding boxes

In [ ]:
img = cv2.imread(Path)
bbx_thickness = (img.shape[0] + img.shape[1]) // 450
count = 0
array = ['build','pool']
for r in results:
    for box in r.boxes:
        textString = ""
#         Extract object class and confidence score
        score = box.conf.item() * 100
        class_id = int(box.cls.item())
        x1 , y1 , x2, y2 = np.squeeze(box.xyxy.numpy()).astype(int)
            # Print detection info
        class_id = box.cls[0].item()
        class_id = round(class_id)

#         print(array[class_id])
        prob = round(box.conf[0].item(), 2)
#         textString += f"{class_id}"
        textString += f"{array[class_id]}"
        prob = prob * 100
        textString += f" {prob}%"

        # Calculate font scale based on object size
        font = cv2.FONT_HERSHEY_COMPLEX
        fontScale = (((x2 - x1) / img.shape[0]) + ((y2 - y1) / img.shape[1])) / 2 * 2.5
        fontThickness = 1
        textSize, baseline = cv2.getTextSize(textString, font, fontScale, fontThickness)

        # Draw bounding box, a centroid and label on the image
        img = cv2.rectangle(img, (x1,y1), (x2,y2), color = (255,0,0),  thickness=bbx_thickness)
        center_coordinates = ((x1 + x2)//2, (y1 + y2) // 2)

        img =  cv2.circle(img, center_coordinates, 5 , (0,0,255), -1)
        if textString != "":
            if (y1 < textSize[1]):
                y1 = y1 + textSize[1]
            else:
                y1 -= 2
            # show the details text in a filled rectangle
#             img = cv2.rectangle(img, (x1, y1), (x1 + textSize[0] , y1 -  textSize[1]), color = (255,0,0), cv2.cv.CV_FILLED == -1)
            img = cv2.rectangle(img, (x1, y1), (x1 + textSize[0] , y1 -  textSize[1]), color = (255,0,0))

            img = cv2.putText(img, textString ,
                (x1, y1), font,
                fontScale,  (0, 0, 0), fontThickness, cv2.LINE_AA)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
Source
# https://docs.ultralytics.com/modes/predict/#working-with-results
# https://www.freecodecamp.org/news/how-to-detect-objects-in-images-using-yolov8/